In [1]:
from pydantic import BaseModel, Field

In [2]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=5)

In [3]:
from typing import Annotated

BoundedInt = Annotated[int, Field(gt=0, lt=5)]

class Model(BaseModel):
    number: BoundedInt

In [4]:
from datetime import datetime
from typing import Any

from dateutil.parser import parse

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

In [5]:
from pydantic import BeforeValidator

In [6]:
DateTime = Annotated[datetime, BeforeValidator(parse_datetime)]

In [7]:
class Model(BaseModel):
    dt: DateTime

In [8]:
Model(dt="2020/1/1 3pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

In [9]:
from pydantic import AfterValidator

In [10]:
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [11]:
DateTime = Annotated[
    datetime,
    BeforeValidator(parse_datetime), 
    AfterValidator(make_utc)
]

In [12]:
class Model(BaseModel):
    dt: DateTime

In [13]:
Model(dt="2020/1/1 3pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [14]:
def before_validator_1(value):
    print("before_validator_1")
    return value

def before_validator_2(value):
    print("before_validator_2")
    return value
    
def before_validator_3(value):
    print("before_validator_3")
    return value

def after_validator_1(value):
    print("after_validator_1")
    return value

def after_validator_2(value):
    print("after_validator_2")
    return value

def after_validator_3(value):
    print("after_validator_3")
    return value

In [15]:
CustomType = Annotated[
    int,
    BeforeValidator(before_validator_1),
    BeforeValidator(before_validator_2),
    BeforeValidator(before_validator_3),
    AfterValidator(after_validator_1),
    AfterValidator(after_validator_2),
    AfterValidator(after_validator_3),
]

In [16]:
class Model(BaseModel):
    number: CustomType

In [17]:
Model(number=10)

before_validator_3
before_validator_2
before_validator_1
after_validator_1
after_validator_2
after_validator_3


Model(number=10)

In [22]:
def are_element_unique(values: list[Any]) -> list[Any]:
    unique_elements: list = []
    for value in values:
        if value in unique_elements:
            raise ValueError("elements must be unique")
        unique_elements.append(value)
    return values

In [23]:
UniqueIntegerList = Annotated[list[int], AfterValidator(are_element_unique)]

In [24]:
class Model(BaseModel):
    numbers: UniqueIntegerList = []

In [26]:
Model(numbers=(1, 2, 3, 4, 5))

Model(numbers=[1, 2, 3, 4, 5])

In [27]:
Model(numbers=[1, 1])

ValidationError: 1 validation error for Model
numbers
  Value error, elements must be unique [type=value_error, input_value=[1, 1], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [28]:
from typing import TypeVar

In [29]:
T = TypeVar("T")

In [31]:
UniqueList = Annotated[list[T], AfterValidator(are_element_unique)]

In [37]:
class Model(BaseModel):
    numbers: UniqueList[int] = []
    strings: UniqueList[str] = []

In [38]:
Model(
    numbers=[1, 2, 3],
    strings=["pyt", "hon"]
)

Model(numbers=[1, 2, 3], strings=['pyt', 'hon'])

In [39]:
Model(numbers=[1, 1])

ValidationError: 1 validation error for Model
numbers
  Value error, elements must be unique [type=value_error, input_value=[1, 1], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [41]:
Model(numbers=["a", 2, 3], strings=["a", 2])

ValidationError: 2 validation errors for Model
numbers.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing
strings.1
  Input should be a valid string [type=string_type, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type

In [46]:
UniqueList = Annotated[
    list[T], 
    Field(min_length=2, max_length=5),
    AfterValidator(are_element_unique)
]

In [47]:
class Model(BaseModel):
    numbers: UniqueList[int] = []
    strings: UniqueList[str] = []

In [48]:
Model(numbers=[1, 2, 3], strings=["a", "b", "c"])

Model(numbers=[1, 2, 3], strings=['a', 'b', 'c'])

In [49]:
Model(numbers=["a"], strings=[1, 2, 3, 4, 5, 6])

ValidationError: 2 validation errors for Model
numbers.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing
strings
  List should have at most 5 items after validation, not 6 [type=too_long, input_value=[1, 2, 3, 4, 5, 6], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/too_long